# MER algorithm for dynamic features

### Load data

In [1]:
import pandas as pd
import os
import numpy as np
from math import sqrt
from tqdm import tqdm

from sklearn.utils import shuffle
from sklearn.linear_model import Lasso, ElasticNet, Ridge
from sklearn.svm import SVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, KFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import make_pipeline
from nltk.stem.snowball import SnowballStemmer
import IPython.display as ipd

In [2]:
audio_features_dir = '/Users/gioelepozzi/Desktop/data/features_thesis/dynamic_features.csv'
audio_features_df = pd.read_csv(audio_features_dir)

eda_features_dir = '/Users/gioelepozzi/Desktop/data/features_thesis/dynamic_features_EDA.csv'
eda_features_df = pd.read_csv(eda_features_dir)
eda_dataset = eda_features_df.groupby(by=['music_ID', 'frame'], as_index=False).mean() # mean over 10 subjects
eda_dataset = eda_dataset.drop(columns=['subject_ID'])

VA_mean_dir = '/Users/gioelepozzi/Desktop/data/annotations_thesis/dynamic_annotations.csv'
VA_std_dir = '/Users/gioelepozzi/Desktop/data/annotations_thesis/dynamic_annotations_std.csv'
VA_mean_df = pd.read_csv(VA_mean_dir)
VA_std_df = pd.read_csv(VA_std_dir)

audio_df = pd.merge(audio_features_df, VA_mean_df, on=['music_ID', 'frame'])
audio_df = pd.merge(audio_df, VA_std_df, on=['music_ID', 'frame']) # df with audio features and VA values
audio_df = audio_df.fillna(0)

eda_df = pd.merge(eda_dataset, VA_mean_df, on=['music_ID', 'frame'])
eda_df = pd.merge(eda_df, VA_std_df, on=['music_ID', 'frame'])

In [3]:
audio_features_v_mean = audio_df.drop(columns=['music_ID', 'frame', 'Arousal(mean)', 'Arousal(std)', 'Valence(std)'])
audio_features_a_mean = audio_df.drop(columns=['music_ID', 'frame', 'Valence(mean)', 'Arousal(std)', 'Valence(std)'])
audio_features_v_std = audio_df.drop(columns=['music_ID', 'frame', 'Arousal(mean)', 'Valence(mean)', 'Arousal(std)'])
audio_features_a_std = audio_df.drop(columns=['music_ID', 'frame', 'Arousal(mean)', 'Valence(mean)', 'Valence(std)'])

eda_features_v_mean = eda_df.drop(columns=['music_ID', 'frame', 'Arousal(mean)', 'Arousal(std)', 'Valence(std)'])
eda_features_a_mean = eda_df.drop(columns=['music_ID', 'frame', 'Valence(mean)', 'Arousal(std)', 'Valence(std)'])
eda_features_v_std = eda_df.drop(columns=['music_ID', 'frame', 'Arousal(mean)', 'Valence(mean)', 'Arousal(std)'])
eda_features_a_std = eda_df.drop(columns=['music_ID', 'frame', 'Arousal(mean)', 'Valence(mean)', 'Valence(std)'])

arousal_mean = audio_df['Arousal(mean)']
valence_mean = audio_df['Valence(mean)']
arousal_std = audio_df['Arousal(std)']
valence_std = audio_df['Valence(std)']

eda_arousal_mean = eda_df['Arousal(mean)']
eda_valence_mean = eda_df['Valence(mean)']
eda_arousal_std = eda_df['Arousal(std)']
eda_valence_std = eda_df['Valence(std)']

In [4]:
fusion_dataset = pd.merge(audio_df, eda_df, on=['music_ID', 'frame'])
fusion_features = fusion_dataset.drop(columns=['music_ID', 'frame'])

In [5]:
def load_audio_dataset(data):
    features = data[data.columns[:-1]].values
    labels = data[data.columns[-1]].values
    #scaler = StandardScaler(copy=False)
    #scaler.fit_transform(features)
    return features, labels

### Regressors

In [6]:
def rmse(y, y_pred):
    return sqrt(mean_squared_error(y, y_pred))

regressors = {
    'Lasso': Lasso(),
    'ElasticNet': ElasticNet(),
    'Ridge': Ridge(),
    'kNN': KNeighborsRegressor(),
    'SVRrbf': SVR(kernel='rbf', gamma='scale'),
    #'SVRpoly': SVR(kernel='poly', gamma='scale'),
    #'SVRlinear': SVR(kernel='linear', gamma='scale'),
    'DT': DecisionTreeRegressor(max_depth=5),
    'RF': RandomForestRegressor(max_depth=5, n_estimators=10, max_features=1),
    #'MLP': MLPRegressor(hidden_layer_sizes=(200,50), max_iter=2000),
    #'AdaBoost': AdaBoostRegressor(n_estimators=10),
}

In [7]:
def cross_val_regression(regressors, features, labels, preprocessfunc):
    columns = list(regressors.keys())
    scores = pd.DataFrame(columns=columns, index=['RMSE'])

    for reg_name, reg in tqdm(regressors.items(), desc='regressors'):
        scorer = {'rmse': make_scorer(rmse)}
        reg = make_pipeline(*preprocessfunc, reg)
        reg_score = cross_validate(reg, features, labels, scoring=scorer, cv=10, return_train_score=False) 
        scores.loc['RMSE', reg_name] = reg_score['test_rmse'].mean()
#         scores.loc['R', reg_name] = reg_score['test_r'].mean()
    return scores

def format_scores(scores):
    def highlight(s):
        is_min = s == min(s)
#         is_max = s == max(s)
#         is_max_or_min = (is_min | is_max)
        return ['background-color: yellow' if v else '' for v in is_min]
    scores = scores.style.apply(highlight, axis=1, subset=pd.IndexSlice[:, :scores.columns[-2]])
    return scores.format('{:.3f}')

def regression_results(regressors, trainset, testset, featureNames, labelName, filePrefix, preprocessfunc):
    X_train = trainset[featureNames]
    y_train = trainset[labelName]
    X_test = testset[featureNames]
    y_test = testset[labelName]

    columns = ['music_ID', 'y_test'] + list(regressors.keys())
    results = pd.DataFrame(columns=columns)
    results['music_ID'] = testset['music_ID']
    results['y_test'] = y_test.values
    
    for reg_name, reg in tqdm(regressors.items(), desc='regressors'):
        reg = make_pipeline(*preprocessfunc, reg)
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        results[reg_name] = y_pred
        results.to_csv(os.path.join('temp_results',f'{filePrefix}_regression_results_{labelName}.csv'))
    
def compute_rmse_across_songs(resultsFile):
    results = pd.read_csv(resultsFile,index_col=0).dropna(axis=1, how='any')
    columns = results.columns[2:]
    scores = pd.DataFrame(columns=columns, index=['rmse_across_segments', 'rmse_across_songs'])
    rmse_across_songs = {}
    testsongs_num = len(results['music_ID'].unique())

    for reg_name in columns:
        scores.loc['rmse_across_segments', reg_name] = rmse(results['y_test'], results[reg_name])
        rmse_across_songs[reg_name] = 0

    for i, g in results.groupby('music_ID'):
        for reg_name in columns:
            rmse_across_songs[reg_name] += rmse(g['y_test'], g[reg_name])

    for reg_name in columns:
        scores.loc['rmse_across_songs', reg_name] = rmse_across_songs[reg_name]/testsongs_num
    
    mean_rmse = scores.mean(axis=1)
    std_rmse = scores.std(axis=1)
    
    scores['Mean'] = mean_rmse
    scores['std'] = std_rmse
    ipd.display(format_scores(scores))

In [8]:
songs = audio_df['music_ID'].unique()
songs = shuffle(songs, random_state=3)
test_num = round(len(songs)*0.1)
testsongs = songs[:test_num]
print(list(testsongs))

[63, 490, 34, 743, 104, 177, 79, 894, 14, 668, 683, 151, 504, 516, 355, 98, 97, 579, 892, 837, 152, 169, 388, 391, 561, 850, 985, 958, 572, 514, 625, 791, 517, 507, 501, 1000, 803, 457, 403, 670, 51, 798, 59, 531, 466, 503, 794, 568, 279, 103, 350, 917, 428, 417, 393, 571, 354, 283, 906, 149, 56, 128, 742, 993, 94, 754, 199, 57, 576, 463, 284, 126, 488, 253, 227, 730, 861]


### Multiple regressors on audio features

In [9]:
iftestset = audio_df['music_ID'].apply(lambda x: x in testsongs)
testset = audio_df[iftestset]
trainset = audio_df[~iftestset]
prefunc = [StandardScaler()]
featureNames = audio_df.columns[2:-4]

print('Audio Features:\n')

print('In Arousal dimension...')
regression_results(regressors, trainset, testset, featureNames, 'Arousal(mean)', 'audio', prefunc)

print('In Valence dimension...')
regression_results(regressors, trainset, testset, featureNames, 'Valence(mean)', 'audio', prefunc)

regressors:   0%|          | 0/7 [00:00<?, ?it/s]

Audio Features:

In Arousal dimension...


regressors:  14%|█▍        | 1/7 [00:00<00:00,  6.74it/s]

In Valence dimension...


regressors: 100%|██████████| 7/7 [01:12<00:00, 10.36s/it]


In [10]:
print('In Arousal dimension...')
compute_rmse_across_songs(os.path.join('temp_results','audio_regression_results_Arousal(mean).csv'))
print('In Valence dimension...')
compute_rmse_across_songs(os.path.join('temp_results','audio_regression_results_Valence(mean).csv'))

In Arousal dimension...


,Lasso,ElasticNet,Ridge,kNN,SVRrbf,DT,RF,Mean,std
rmse_across_segments,0.187,0.187,0.120,0.116,0.103,0.118,0.145,0.140,0.034
rmse_across_songs,0.158,0.158,0.085,0.104,0.085,0.095,0.118,0.115,0.032


In Valence dimension...


,Lasso,ElasticNet,Ridge,kNN,SVRrbf,DT,RF,Mean,std
rmse_across_segments,0.157,0.157,0.140,0.135,0.125,0.128,0.130,0.139,0.013
rmse_across_songs,0.128,0.128,0.105,0.112,0.100,0.104,0.104,0.111,0.012


### Multiple regressors on EDA features

In [11]:
iftestset = eda_df['music_ID'].apply(lambda x: x in testsongs)
testset = eda_df[iftestset]
trainset = eda_df[~iftestset]
prefunc = [StandardScaler()]
featureNames = list(set(eda_df.columns).difference({'subject_ID', 'music_ID', 'frame', 
                                                         'Arousal(mean)', 'Valence(mean)'}))

print('In Arousal dimension...')
regression_results(regressors, trainset, testset, featureNames, 'Arousal(mean)', 'eda', prefunc)

print('In Valence dimension...')
regression_results(regressors, trainset, testset, featureNames, 'Valence(mean)', 'eda', prefunc)

regressors:  29%|██▊       | 2/7 [00:00<00:00, 15.74it/s]

In Arousal dimension...


regressors:  29%|██▊       | 2/7 [00:00<00:00, 15.75it/s]

In Valence dimension...


regressors: 100%|██████████| 7/7 [00:21<00:00,  3.01s/it]


In [12]:
print('In Arousal dimension...')
compute_rmse_across_songs(os.path.join('temp_results','eda_regression_results_Arousal(mean).csv'))
print('In Valence dimension...')
compute_rmse_across_songs(os.path.join('temp_results','eda_regression_results_Valence(mean).csv'))

In Arousal dimension...


,Lasso,ElasticNet,Ridge,kNN,SVRrbf,DT,RF,Mean,std
rmse_across_segments,0.181,0.181,0.147,0.172,0.146,0.150,0.174,0.164,0.016
rmse_across_songs,0.151,0.151,0.113,0.152,0.121,0.129,0.146,0.137,0.016


In Valence dimension...


,Lasso,ElasticNet,Ridge,kNN,SVRrbf,DT,RF,Mean,std
rmse_across_segments,0.152,0.152,0.118,0.138,0.117,0.121,0.150,0.135,0.016
rmse_across_songs,0.123,0.123,0.102,0.126,0.105,0.106,0.122,0.115,0.010


### Multiple regressors on audio + EDA features

In [20]:
iftestset = fusion_dataset['music_ID'].apply(lambda x: x in testsongs)
testset = fusion_dataset[iftestset]
trainset = fusion_dataset[~iftestset]
prefunc = [StandardScaler()]
featureNames = list(set(fusion_dataset.columns).difference({'subject_ID', 'music_ID', 'frame', 
                                                         'Arousal(mean)_y', 'Valence(mean)_y'}))
print('In Arousal dimension...')
regression_results(regressors, trainset, testset, featureNames, 'Arousal(mean)_y', 'mean_fusion', prefunc)

print('In Valence dimension...')
regression_results(regressors, trainset, testset, featureNames, 'Valence(mean)_y', 'mean_fusion', prefunc)

regressors:  14%|█▍        | 1/7 [00:00<00:00,  6.80it/s]

In Arousal dimension...


regressors:  14%|█▍        | 1/7 [00:00<00:00,  7.26it/s]

In Valence dimension...


regressors: 100%|██████████| 7/7 [00:14<00:00,  2.11s/it]


In [22]:
print('In Arousal dimension...')
compute_rmse_across_songs(os.path.join('temp_results','mean_fusion_regression_results_Arousal(mean)_y.csv'))
print('In Valence dimension...')
compute_rmse_across_songs(os.path.join('temp_results','mean_fusion_regression_results_Valence(mean)_y.csv'))

In Arousal dimension...


,Lasso,ElasticNet,Ridge,kNN,SVRrbf,DT,RF,Mean,std
rmse_across_segments,0.181,0.181,0.000,0.101,0.069,0.008,0.139,0.097,0.075
rmse_across_songs,0.151,0.151,0.000,0.093,0.057,0.007,0.115,0.082,0.063


In Valence dimension...


,Lasso,ElasticNet,Ridge,kNN,SVRrbf,DT,RF,Mean,std
rmse_across_segments,0.152,0.152,0.000,0.115,0.085,0.006,0.136,0.092,0.065
rmse_across_songs,0.123,0.123,0.000,0.097,0.068,0.006,0.109,0.075,0.053
